In [17]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset
import torch
import numpy as np

import onnxruntime


In [36]:
# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-large")
model = onnxruntime.InferenceSession("onnx/model.onnx")
model_pt = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

In [38]:
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
input_features = processor(ds[0]["audio"]["array"], return_tensors="np").input_features 
input_features_pt = processor(ds[0]["audio"]["array"], return_tensors="pt").input_features 


Reusing dataset librispeech_asr (/home/vladimir/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b)
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [25]:
input_features.shape

(1, 80, 3000)

In [26]:
[inp.name for inp in model.get_inputs()]

['input_features', 'decoder_input_ids']

In [42]:
%timeit -r 100 logits = model.run(output_names=["last_hidden_state"], input_feed={"input_features": input_features, "decoder_input_ids": np.array([[50258]])}) 

341 ms ± 7.87 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [43]:
%timeit -r 100 model_pt(input_features_pt, decoder_input_ids = torch.tensor([[50258]]))

512 ms ± 113 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [31]:
logits[0].shape

(1, 1, 512)